In [ ]:
# install the requirements
!pip install spacy
!python -m spacy download es_core_news_md
!python -m spacy download es_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 78.4 MB 6.8 MB/s 
  Created wheel for es-core-news-md: filename=es_core_news_md-2.2.5-py3-none-any.whl size=79649484 sha256=51872de942c0a6670c34bf1f8ed3bc171bf3ec4209f59773d468f5516046ee60
  Stored in directory: /tmp/pip-ephem-wheel-cache-re1j0o85/wheels/d8/f5/92/ee8a4f74fac67775fbc0314b1c9ae4694f4180437f6fc3dd1c
Successfully built es-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.2 MB 5.2 MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-py3-none-any.whl size=16172933 sha256=76f25710fc5befdc687d2a2b3b55ba493a3e5eb5de2f5a3f2

In [ ]:
!pip install stanza
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 5.2 MB/s 
     |████████████████████████████████| 4.2 MB 33.0 MB/s 
     |████████████████████████████████| 175 kB 49.8 MB/s 
     |████████████████████████████████| 596 kB 39.1 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 42.7 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=5134f742240bf69b104a5b89957e2bf644bd5a52a159575c32d4ed8155e3cf6c
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import pandas as pd
import es_core_news_md
import es_core_news_sm
from IPython.display import SVG
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download("punkt")

pd.options.display.max_rows = 600
pd.options.display.max_colwidth = 400

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import emoji
def give_emoji_free_text(text):
    return emoji.replace_emoji(text, replace='')

In [ ]:
#Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/Shareddrives/CrimeTrack: Tracking Crime on the Web/Data/Train Dataset/named_entity_training.xlsx'

In [ ]:
file = pd.read_excel(path)

In [ ]:
file.head()

,username,creation_date,tweet,likes,retweets,crime_label,crime_type,location
0,policia,2019-10-15 12:30:37,🚩ÚLTIMA HORA\n\nLa @policia Nacional ha detenido a la persona que agredió ayer a una ciudadana de Tarragona por portar una bandera española\n\nHa sido arrestado por delito contra el ejercicio de los Derechos Fundamentales y Libertades Públicas y delito leve de lesiones\n#EstamosPorTi https://t.co/DnDQAYkTyd,45938.0,16295.0,Crime,Injury,Tarragona
1,policia,2016-07-07 21:08:09,"Oiga @policia, mi perro ha sido agredido sexualmente por un Ponyta salvaje. ¿Qué se hace en estos casos? https://t.co/nn9BhbTnE9",27453.0,19675.0,Crime,Sexual Violence,NaN
2,policia,2020-09-11 12:48:39,"🚩Identificadas las tres menores, dos de 15 años y una de ellas de 16, protagonistas de la agresión racista en el Metro de Madrid.\n\nLa Brigada Móvil de @policia ya ha dado cuenta a la Fiscalía de Menores por un presunto delito de odio.\n\nStop #DelitosDeOdio \n#Respeto",27089.0,7233.0,Crime,Injury,Metro de Madrid
3,policia,2020-06-08 07:08:00,"🚨Un padre de familia sustrae 4 potitos en un supermercado de #Zaragoza. Es sorprendido y llaman📞a la @policia \n\nEl agente👮‍♂️ que acude, viendo la necesidad del hombre, paga la comida infantil y lo deja ir\n\n#MicrocuentosReales\n#FelizLunes\n#SomosTuPolicía \n\nhttps://t.co/VsZF1vgDsg https://t.co/gUmfKr10H4",25674.0,7050.0,Crime,Fraud,Zaragoza
4,policia,2021-07-19 17:30:56,🚩Esta tarde ha sido detenido el presunto autor de la #agresión a un sanitario ocurrida en #MetroDeMadrid 📹⤵ https://t.co/zqWUV3v7nk,25041.0,5167.0,Crime,Injury,Metro de Madrid


Spacy

In [ ]:
nlp_1 = es_core_news_md.load()
nlp_2 = es_core_news_sm.load()

In [ ]:
document = nlp_1(file.tweet[0])

In [ ]:
file['loc_spacy'] = ''
for i in range(len(file)):
  document = nlp_1(give_emoji_free_text(file.tweet[i]))
  loc = []
  for named_entity in document.ents:
    if named_entity.label_ == 'LOC':
      loc.append(str(named_entity))
  file['loc_spacy'][i] = ",".join(loc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
file['loc_spacy_1'] = ''

for i in range(len(file)):
  document = nlp_2(give_emoji_free_text(file.tweet[i]))
  loc = []
  for named_entity in document.ents:
    if named_entity.label_ == 'LOC':
      loc.append(str(named_entity))
  file['loc_spacy_1'][i] = ",".join(loc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
from spacy import displacy

def display_entities(i):
  print('ORIGINAL TWEET\n')
  print(file.tweet[i], '\n')
  tweet = give_emoji_free_text(file.tweet[i].lower())
  tweet = tweet.replace("¡", "")
  document = nlp_1(tweet)
  doc = {'text': tweet, 'ents': []}
  for named_entity in document.ents:
      #print(named_entity, named_entity.label_)
      
      index = tweet.lower().index(str(named_entity))
      doc['ents'].append({'start':index, 'end': index + len(str(named_entity)), 'label': named_entity.label_})
  labels = []
  for i in range(len(doc['ents'])): 
    labels.append(doc['ents'][i]['label'])
  tags = set(labels)
  colors = ["#9cc9cc", "#bfeeb7", "#feca74", "#ff9561", "#aa9cfc", "#c887fb", "#9cc9cc", "#ffeb80", "#ff8197", "#ff8197", "#f0d0ff", "#bfe1d9", "#bfe1d9", "#e4e7d2", "#e4e7d2", "#e4e7d2", "#e4e7d2", "#e4e7d2"]
  tags_colors = {tag: color for tag, color in zip(tags, colors)}
  options = {'ents': tags, 'colors': tags_colors}
  print('###################\n')
  displacy.render(doc, style="ent", manual=True, jupyter=True,options=options)

In [ ]:
display_entities(10)

ORIGINAL TWEET

🚩 ÚLTIMA HORA: Detenido en Marruecos un yihadista que presuntamente pretendía atentar en #Sevilla. 
En estos momentos se está llevando a cabo el registro en su domicilio en la capital andaluza #EstamosPorTi #SomosTuPolicía 

###################



Stanza

In [ ]:
import stanza
stanza.download('es') # download Spanish model
nlp = stanza.Pipeline(lang='es', processors='tokenize,ner') # initialize Spanish neural pipeline

2022-05-25 16:45:39 INFO: Downloading default packages for language: es (Spanish)...


2022-05-25 16:45:52 INFO: Finished downloading models and saved to /root/stanza_resources.


2022-05-25 16:45:52 WARNING: Language es package default expects mwt, which has been added
2022-05-25 16:45:53 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| ner       | conll02 |

2022-05-25 16:45:53 INFO: Use device: cpu
2022-05-25 16:45:53 INFO: Loading: tokenize
2022-05-25 16:45:53 INFO: Loading: mwt
2022-05-25 16:45:53 INFO: Loading: ner
2022-05-25 16:45:55 INFO: Done loading processors!


In [ ]:
file['loc_stanza'] = ''

for i in range(len(file)):
  document = nlp(give_emoji_free_text(file.tweet[i]))
  loc = []
  for named_entity in document.ents:
    if named_entity.type == 'LOC':
      loc.append(named_entity.text)
  file['loc_stanza'][i] = ",".join(loc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


BERT

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, FillMaskPipeline
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/roberta-base-bne-capitel-ner-plus") #MMG/xlm-roberta-large-ner-spanish
model = AutoModelForTokenClassification.from_pretrained("PlanTL-GOB-ES/roberta-base-bne-capitel-ner-plus")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/831k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/497k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

In [ ]:
def ber_ner(index):
  sentence = give_emoji_free_text(file.tweet[index])
  ner_results = nlp(sentence)
  doc = {'text': sentence, 'ents': []}
  count=0
  same_entity = False
  previous_entity = ''
  last_start = 0
  for i in range(len(ner_results)): 
    if 'LOC' not in ner_results[i]['entity']:
      continue
    else: 
      if ner_results[i]['entity'].startswith('B') or ner_results[i]['entity'].startswith('S'): 
        doc['ents'].append({'word':sentence[ner_results[i]['start']: ner_results[i]['end']] ,'label': ner_results[i]['entity'].replace('B_', '').replace('S_', '')})
        same_entity = True
        previous_entity = ner_results[i]['entity'].replace('B_', '').replace('S_', '')
        last_index =  ner_results[i]['start']
      elif ner_results[i]['entity'].startswith('I') or ner_results[i]['entity'].startswith('E'): 
        if ner_results[i]['entity'].replace('I_', '').replace('E_', '') != previous_entity:
          print(ner_results[i]['entity'].replace('I_', '').replace('E_', ''))
          same_entity = False
        if same_entity:
          doc['ents'][-1] = {'word':sentence[last_index: ner_results[i]['end']], 'label': previous_entity}
  return doc['ents']

In [ ]:
file['loc_bert'] = ''
for i in range(len(file)):
    pred = ber_ner(i)
    loc = []
    for j in range(len(pred)):
      if pred[j]['word'] != '#':
        loc.append(pred[j]['word'])
    file['loc_bert'][i] = ",".join(loc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
from spacy import displacy

def display(sentence, ner_results):
  doc = {'text': sentence, 'ents': []}
  count=0
  same_entity = False
  previous_entity = ''
  last_start = 0
  for i in range(len(ner_results)): 
    if 'MISC' in ner_results[i]['entity']:
      continue
    else: 
      if ner_results[i]['entity'].startswith('B'): 
        doc['ents'].append({'start':ner_results[i]['start'], 'end': ner_results[i]['end'], 'label': ner_results[i]['entity'].replace('B-', '')})
        same_entity = True
        previous_entity = ner_results[i]['entity'].replace('B-', '')
        last_start = ner_results[i]['start']
      elif ner_results[i]['entity'].startswith('I'): 
        if ner_results[i]['entity'].replace('I-', '') != previous_entity:
          same_entity = False
        if same_entity:
          doc['ents'][-1] = {'end':ner_results[i]['end'], 'label': previous_entity, 'start':last_start}

  tags = set(['LOC','MISC','ORG','PER'])
  colors = ["#7aecec", "#bfeeb7", "#feca74", "#ff9561", "#aa9cfc", "#c887fb", "#9cc9cc", "#ffeb80", "#ff8197", "#ff8197", "#f0d0ff", "#bfe1d9", "#bfe1d9", "#e4e7d2", "#e4e7d2", "#e4e7d2", "#e4e7d2", "#e4e7d2"]
  tags_colors = {tag: color for tag, color in zip(tags, colors)}
  options = {'ents': tags, 'colors': tags_colors}

  displacy.render(doc, style="ent", manual=True, jupyter=True, options=options)


In [ ]:
file.loc[file.loc[file.location.isnull()].index,'location'] = ''

In [ ]:
def precision_recall(file, column):
  true_positive = 0
  false_positive = 0
  false_negative = 0
  for i in range(len(file)):
    loc = list(file.location[i].split(','))
    #print(loc)
    pred = list(file[column][i].split(','))
    #print(pred)
    for i in range(len(loc)):
      for j in range(len(pred)):
        if pred[j] == loc[i]:
          true_positive += 1
          #print(pred[j],loc[i])
    for j in range(len(pred)):
      if  pred[j] not in loc: 
        false_positive += 1
        #print(pred[j])
    for j in range(len(loc)):
      if  loc[j] not in pred: 
        false_negative += 1
        #print(loc[j])
  precision = true_positive / (true_positive + false_positive)
  recall = true_positive / (true_positive + false_negative)
  return precision, recall

In [ ]:
precision_stanza, recall_stanza = precision_recall(file, 'loc_stanza')
print('STANZA')
print('Precision:', precision_stanza)
print('Recall:', recall_stanza)
print('F1:', (2*precision_stanza*recall_stanza)/(recall_stanza+precision_stanza))


STANZA
Precision: 0.7571428571428571
Recall: 0.8548387096774194
F1: 0.803030303030303


In [ ]:
precision_bert, recall_bert = precision_recall(file, 'loc_bert')
print('BERT')
print('Precision:', precision_bert)
print('Recall:', recall_bert)
print('F1:', (2*precision_bert*recall_bert)/(precision_bert+recall_bert))


BERT
Precision: 0.5783132530120482
Recall: 0.7741935483870968
F1: 0.6620689655172414


In [ ]:
precision_spacy, recall_spacy = precision_recall(file, 'loc_spacy')
print('SPACY MD')
print('Precision:', precision_spacy)
print('Recall:', recall_spacy)
print('F1:', (2*precision_spacy*recall_spacy)/(precision_spacy+recall_spacy))


SPACY MD
Precision: 0.5308641975308642
Recall: 0.6935483870967742
F1: 0.6013986013986015


In [ ]:
precision_spacy_1, recall_spacy_1 = precision_recall(file, 'loc_spacy_1')
print('SPACY SM')
print('Precision:', precision_spacy_1)
print('Recall:', recall_spacy_1)
print('F1:', (2*precision_spacy_1*recall_spacy_1)/(precision_spacy_1+recall_spacy_1))


SPACY SM
Precision: 0.4819277108433735
Recall: 0.6451612903225806
F1: 0.5517241379310345
